In [ ]:
import pygame
import numpy as np
from random import randrange as rnd

WIDTH, HEIGHT = 1000, 600
fps = 60   

# paddle settings
paddle_w = 200
paddle_h = 20
paddle = pygame.Rect(WIDTH // 2 - paddle_w // 2, HEIGHT - paddle_h - 10, paddle_w, paddle_h)  


# ball settings
ball_radius = 20
ball_speed = 5
ball_rect = int(ball_radius * 2 ** 0.5)  ##??
dx, dy = 1, -1 

# blocks settings
block_list = [pygame.Rect(10 + 120 * i, 10 + 70 * j, 100, 50) for i in range(8) for j in range(4)]
color_list = [(rnd(30, 256), rnd(30, 256), rnd(30, 256)) for i in range(8) for j in range(4)]

# Initialize the game board using NumPy
game_board = np.zeros((HEIGHT, WIDTH), dtype=int)

pygame.init() 
sc = pygame.display.set_mode((WIDTH, HEIGHT)) 
clock = pygame.time.Clock()  


# background image
img = pygame.image.load('1.jpg').convert()

game_over = True  

def reset_game(): 
    global paddle, ball, dx, dy, block_list, color_list, game_over 
  
    paddle = pygame.Rect(WIDTH // 2 - paddle_w // 2, HEIGHT - paddle_h - 10, paddle_w, paddle_h)
    ball = pygame.Rect(rnd(ball_rect, WIDTH - ball_rect), HEIGHT // 2, ball_rect, ball_rect)
    dx, dy = 1, -1
    block_list = [pygame.Rect(10 + 120 * i, 10 + 70 * j, 100, 50) for i in range(8) for j in range(4)]
    color_list = [(rnd(30, 256), rnd(30, 256), rnd(30, 256)) for i in range(8) for j in range(4)]
    game_over = False

def detect_collision(dx, dy, ball, rect):
    if dx > 0:
        delta_x = ball.right - rect.left  
    else:
        delta_x = rect.right - ball.left
    if dy > 0:
        delta_y = ball.bottom - rect.top 
    else:
        delta_y = rect.bottom - ball.top

    if abs(delta_x - delta_y) < 10:
        dx, dy = -dx, -dy
    elif delta_x > delta_y:
        dy = -dy
    elif delta_y > delta_x:
        dx = -dx
    return dx, dy

reset_game()  # Initialize the game

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            exit()
        if game_over and event.type == pygame.MOUSEBUTTONDOWN:
            reset_game()  # Reset the game on click

    sc.blit(img, (0, 0))
    if not game_over:
        # drawing world
        [pygame.draw.rect(sc, color_list[color], block) for color, block in enumerate(block_list)]
        pygame.draw.rect(sc, pygame.Color('darkorange'), paddle)
        pygame.draw.circle(sc, pygame.Color('white'), ball.center, ball_radius)
        # ball movement
        ball.x += ball_speed * dx
        ball.y += ball_speed * dy
        # collision left right
        if ball.centerx < ball_radius or ball.centerx > WIDTH - ball_radius:
            dx = -dx
        # collision top
        if ball.centery < ball_radius:
            dy = -dy
        # collision paddle
        if ball.colliderect(paddle) and dy > 0:
            dx, dy = detect_collision(dx, dy, ball, paddle)
        # collision blocks
        hit_index = ball.collidelist(block_list)
        if hit_index != -1:
            hit_rect = block_list.pop(hit_index)
            hit_color = color_list.pop(hit_index)
            dx, dy = detect_collision(dx, dy, ball, hit_rect)
            # special effect
            hit_rect.inflate_ip(ball.width * 3, ball.height * 3)
            pygame.draw.rect(sc, hit_color, hit_rect)
            fps += 2
        # game over condition
        if ball.bottom > HEIGHT:
            game_over = True
        elif not len(block_list):
            game_over = True
        # control
        # Get the mouse's x-coordinate
        mouse_x, _ = pygame.mouse.get_pos()
        # Update the paddle's position based on the mouse
        paddle.centerx = mouse_x
        # Update the game board using NumPy
        game_board.fill(0)
        for block in block_list:
            x, y, w, h = block
            game_board[y:y+h, x:x+w] = 1
    else:
        # Display game over message and instructions to restart
        font = pygame.font.Font(None, 36)
        text = font.render("Game Over - Click to Restart", True, (255, 255, 255))
        text_rect = text.get_rect(center=(WIDTH // 2, HEIGHT // 2))
        sc.blit(text, text_rect)

    # update screen
    pygame.display.flip()
    clock.tick(fps)

